In [ ]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch
from datasets import load_dataset

class NERPredictor:
    def __init__(self, model_dir):
        self.tokenizer = AutoTokenizer.from_pretrained(model_dir)
        self.model = AutoModelForTokenClassification.from_pretrained(model_dir)
        self.model.eval()  # Set the model to evaluation mode

    def predict(self, text):
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, padding=True)
        with torch.no_grad():
            outputs = self.model(**inputs)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        return predictions

def main():
    model_dir = "results/checkpoint-2618" 
    data=load_dataset('DFKI-SLT/cross_ner','ai')

    predictor = NERPredictor(model_dir)

    
    input_text = ("Gotta get that anger up outcha champ….It’s honestly sad. In the meantime…. Please enjoy MY MOVIE TRAILER to my next film “LIFT” which will be dropping on  @netflix in 8 days!!!!! There is a moment in the trailer where @gugumbatharaw says “They Really Love You” ….I now know she’s talking about “Katt” 😂😂😂 Mark yo calendars world!!!! This one is SPECIAL!!!!")

    predictions = predictor.predict(input_text)

    entity_labels = [f"LABEL_{pred}" for pred in predictions[0].numpy()]
    label_names = data["train"].features["ner_tags"].feature.names
    label_dict = {f'LABEL_{i}': label for i, label in enumerate(label_names)}
    label=[]
    for i in entity_labels:
        if i!="LABEL_0":
            label.append(label_dict[i].split("-")[1])
    print(list(set(label)))
    
if __name__ == "__main__":
    main()
